# Oanda Demo Trading Notebook

## Packages & Tools

In [1]:
from utils.packages import *
from utils.tools import *

## Tools

In [2]:
def make_order(accountID, stop_price, instrument, units):
    stopLossOnFill = StopLossDetails(price=stop_price)

    ordr = MarketOrderRequest(
        instrument = instrument,
        units=units,
        stopLossOnFill=stopLossOnFill.data)

    r = orders.OrderCreate(accountID, data=ordr.data)
    rv = api.request(r)
    return(rv)

In [3]:
def close_order(accountID, order_type, instrument):
    data_long = {"longUnits": "ALL"}
    data_short = {"shortUnits": "ALL"}
    
    if order_type == 'long':
        data = data_long
    elif order_type == 'short':
        data = data_short
        
    r = positions.PositionClose(accountID=accountID,
                                instrument=instrument,
                                data=data)
    rv = api.request(r)
    return(rv)

In [4]:
def get_weights(s):
    n = 1/s
    val = 0
    val_list = []
    for i in range(s):
        val += n
        val_list.append(val)
    val_list = np.array(val_list)
    tot = sum(val_list)
    val_list = val_list/tot
    return(val_list)

## API Setup

Read from config file

In [5]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currencies = yaml.load(temp_file)

temp_file = 'config/stepped_loss.yaml'
with open(temp_file) as temp_file:
    loss_limits = yaml.load(temp_file)
    
temp_file = 'config/chart_params.yaml'
with open(temp_file) as temp_file:
    chart_params = yaml.load(temp_file)    

In [6]:
access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']
api = API(access_token = access_token)

## Code Engine

# Log these inputs as well

In [7]:
'''
instrument="EUR_USD"
pip_size = currencies['currs'][instrument]['pip_size']
pip_gap = currencies['currs'][instrument]['pip_gap']

profit_target_num  = 0.5
price_allowed_buffer = 0.5 
loss_limit_num = 25


target_num = 3
min_count = 2
min_count_mulitplier = 1


iter_num = 300
dir_min_trans_multiplier_long = 1
dir_min_trans_multiplier_short = 0.5

run_min_trans_multiplier = 25
dir_recalc_multiplier = 5

num_of_bets = 10'''

'\ninstrument="EUR_USD"\npip_size = currencies[\'currs\'][instrument][\'pip_size\']\npip_gap = currencies[\'currs\'][instrument][\'pip_gap\']\n\nprofit_target_num  = 0.5\nprice_allowed_buffer = 0.5 \nloss_limit_num = 25\n\n\ntarget_num = 3\nmin_count = 2\nmin_count_mulitplier = 1\n\n\niter_num = 300\ndir_min_trans_multiplier_long = 1\ndir_min_trans_multiplier_short = 0.5\n\nrun_min_trans_multiplier = 25\ndir_recalc_multiplier = 5\n\nnum_of_bets = 10'

In [11]:
def run_wma(api, instrument, short_moving_window, long_moving_window, lists_size):
    params = {'instruments': instrument}
    order_flag = 'not_ordered'
    r = pricing.PricingStream(accountID=accountID, params=params)
    rv = api.request(r)

    short_wma = 0
    short_flag = True
    short_list = collections.deque([])
    short_wma_list = collections.deque([])
    short_weights = get_weights(short_moving_window)

    long_wma = 0
    long_flag = True
    long_list = collections.deque([])
    long_wma_list = collections.deque([])
    long_weights = get_weights(long_moving_window)

    tick_list = collections.deque([])

    long_list_counter = 0
    list_ready = False
    old_direction =  'none'
    buy_price = 0
    
    initial_position_captured = False
    
    for i, resp in tqdm(enumerate(rv)):        
        resp_type = resp['type']       

        if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
            pass
        
        #-----------------------------------------------------------------------------------------------------
        elif resp_type == 'PRICE' and order_flag == 'not_ordered': # Check if we are yet to make the order               
            date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
            sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response  
            
            
            #Short list prep
            #-----------------------------------------------------------------------------------------------------
            if len(short_list) < short_moving_window:
                short_list.append(tick_price)

            if short_flag and len(short_list) == short_moving_window:
                short_wma = sum(np.array(short_list)*short_weights)
                short_flag =False

            if short_flag == False and len(short_list) == short_moving_window:
                short_list.popleft()   
                short_list.append(tick_price) 
                short_wma = sum(np.array(short_list)*short_weights)
                

                
            #Long list prep   
            #-----------------------------------------------------------------------------------------------------                
            if len(long_list) < long_moving_window:
                long_list.append(tick_price)

            if long_flag and len(long_list) == long_moving_window:
                long_wma = sum(np.array(long_list)*long_weights)
                long_flag = False

            if long_flag == False and len(long_list) == long_moving_window:
                long_list.popleft()   
                long_list.append(tick_price) 
                long_wma = sum(np.array(long_list)*long_weights)   
                long_list_counter += 1
                if long_list_counter >= lists_size:
                    list_ready = True
             
            
                
            #
            #----------------------------------------------------------------------------------------------------- 
            if list_ready:
                if short_wma > long_wma:
                    current_position = 'positive'
                elif short_wma < long_wma:
                    current_position = 'negative'
                elif short_wma == long_wma:
                    current_position = 'same level'
            
            if list_ready and initial_position_captured == False and current_position != 'same level':
                    initial_position = current_position
                    print(f'initial_position : {initial_position}')
                    initial_position_captured =  True
                
            if list_ready and initial_position_captured:
                if initial_position == current_position:
                    pass
                
                elif current_position == 'same level':
                    print('Same level, wait for next iteration')
                    print(f'long_wma:{long_wma} -- short_wma:{short_wma}')
                    
                elif initial_position != current_position and current_position != 'same level':
                    print(f'current_position:{current_position}')
                    print(f'long_wma:{long_wma} -- short_wma:{short_wma}')
                    
                    if current_position == 'positive':
                        print('Closing open positions if any')
                        print("Going Long")
                        initial_position = current_position
                        profit = (short_wma - buy_price)*1000 
                        print(f'profit:{profit}')
                        buy_price = short_wma
                    
                    elif current_position == 'negative':
                        print('Closing open positions if any')
                        print("Going Short")
                        initial_position = current_position
                        profit = (buy_price - short_wma)*1000 
                        print(f'profit:{profit}')
                        buy_price = short_wma            
                
    #winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)   
    return(short_wma_list, long_wma_list, tick_list)

In [ ]:
short_wma_list, long_wma_list, tick_list = run_wma(api = api,
                                                   instrument="EUR_USD", 
                                                   short_moving_window = 10, 
                                                   long_moving_window = 10, 
                                                   lists_size = 10)

37it [00:20,  2.43it/s]